In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [55]:
df_train= pd.read_csv("train.csv",index_col=[0], parse_dates=[0])
df_test= pd.read_csv("test.csv",index_col=[0], parse_dates=[0])

In [56]:
df_train.drop(df_train.tail(1).index,inplace=True)

In [4]:
df_train.shape

(7110, 11)

In [57]:
df = pd.concat([df_train,df_test])

In [7]:
df.shape

(9357, 11)

In [ ]:
df_train.info()

In [15]:
def create_ml_features(df):   
    
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    df["working_hours"] =  df["hour"].isin(np.arange(8, 21, 1)).astype("int")
    df["is_weekend"] = (df["date"].dt.dayofweek >= 5).astype("int")
    
    df["SMC"] = (df["absolute_humidity"] * 100) / df["relative_humidity"]
    
    # A list of features to generate shifted and lagged values for
    shift_features = ["deg_C", "absolute_humidity", "relative_humidity",
                      "sensor_1", "sensor_2", "sensor_3", "sensor_4", "sensor_5"]
    # Amounts of hour shifts and lags
    shifts = [1, 3, 6, 12, 24]
    
    for feature in shift_features:
        for shift in shifts:
            df[feature+"-"+str(shift)+"abs_shfit"] = df[feature] - df[feature].shift(periods=shift, fill_value=0)
            df[feature+"+"+str(shift)+"abs_shfit"] = df[feature] - df[feature].shift(periods=-shift, fill_value=0)
            df[feature+"-"+str(shift)+"prc_shfit"] = (df[feature] / df[feature].shift(periods=shift, fill_value=0)) - 1
            df[feature+"+"+str(shift)+"prc_shfit"] = (df[feature] / df[feature].shift(periods=-shift, fill_value=0)) - 1

    # Replacing infinity values as a result of devision by zero at the end of a dataset
    df.replace(to_replace=np.inf, value=0, inplace=True)
    
    
    return df.drop('date',axis=1)

In [16]:
df1= create_ml_features(df.copy())

c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  # This is added back by InteractiveShellApp.init_path()
c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\ipykernel_launcher.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\ipykernel_launcher.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\ipykernel

In [58]:
df1= df.copy()

In [59]:
df1.shape

(9357, 11)

In [60]:
df1.reset_index(drop=False, inplace=True)

In [61]:
df1['date_time'] = df1['date_time'].view('datetime64[ns]').view(np.int64)/10**9

In [62]:
df1.shape

(9357, 12)

In [21]:
df1

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,...,sensor_5-6prc_shfit,sensor_5+6prc_shfit,sensor_5-12abs_shfit,sensor_5+12abs_shfit,sensor_5-12prc_shfit,sensor_5+12prc_shfit,sensor_5-24abs_shfit,sensor_5+24abs_shfit,sensor_5-24prc_shfit,sensor_5+24prc_shfit
0,1.268244e+09,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,...,0.000000,0.680613,1293.4,797.8,0.000000,1.609766,1293.4,-73.3,0.000000,-0.053633
1,1.268248e+09,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,...,0.000000,0.413056,1010.9,324.7,0.000000,0.473186,1010.9,-573.8,0.000000,-0.362087
2,1.268251e+09,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,...,0.000000,0.699635,1117.0,70.1,0.000000,0.066960,1117.0,-421.2,0.000000,-0.273827
3,1.268255e+09,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,...,0.000000,1.572709,1263.2,194.1,0.000000,0.181555,1263.2,42.4,0.000000,0.034731
4,1.268258e+09,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,...,0.000000,1.623262,1132.2,332.4,0.000000,0.415604,1132.2,319.4,0.000000,0.392963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,1.301911e+09,23.2,28.7,0.7568,1340.3,1023.9,522.8,1374.0,1659.8,NaN,...,2.649516,0.000000,615.4,1659.8,0.589238,0.000000,1022.3,1659.8,1.603608,0.000000
9353,1.301915e+09,24.5,22.5,0.7119,1232.8,955.1,616.1,1226.1,1269.0,NaN,...,1.311054,0.000000,157.1,1269.0,0.141290,0.000000,661.0,1269.0,1.087171,0.000000
9354,1.301918e+09,26.6,19.0,0.6406,1187.7,1052.4,572.8,1253.4,1081.1,NaN,...,0.284577,0.000000,232.5,1081.1,0.273981,0.000000,418.5,1081.1,0.631603,0.000000
9355,1.301922e+09,29.1,12.7,0.5139,1053.2,1009.0,702.0,1009.8,808.5,NaN,...,-0.466442,0.000000,222.5,808.5,0.379693,0.000000,366.2,808.5,0.827945,0.000000


In [63]:
X_train= df1.iloc[:7110,:]
X_test = df1.iloc[7110:,:]
y = df1[['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]

In [64]:
X_train.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'],axis=1,inplace=True)
X_test.drop(['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'],axis=1,inplace=True)

c:\users\suraj\anaconda3\envs\tabular_july\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [65]:
y_train = y[:7110].copy()

In [66]:
y_train.shape

(7110, 3)

In [67]:
X_train.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,1.268244e+09,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4
1,1.268248e+09,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9
2,1.268251e+09,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0
3,1.268255e+09,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2
4,1.268258e+09,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2


In [68]:
X_train.tail()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
7105,1.293822e+09,9.1,34.3,0.3958,957.9,741.9,970.3,915.1,866.0
7106,1.293826e+09,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5
7107,1.293829e+09,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4
7108,1.293833e+09,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1
7109,1.293836e+09,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3


In [69]:
X_train.shape

(7110, 9)

In [70]:
X_test.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
7110,1.293840e+09,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
7111,1.293844e+09,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
7112,1.293847e+09,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
7113,1.293851e+09,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
7114,1.293854e+09,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [71]:
y_train

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2.5,12.0,167.7
1,2.1,9.9,98.9
2,2.2,9.2,127.1
3,2.2,9.7,177.2
4,1.5,6.4,121.8
...,...,...,...
7105,1.2,4.9,211.0
7106,1.3,5.1,191.1
7107,1.4,5.8,221.3
7108,1.6,5.2,227.4


In [72]:
y_train_log =np.log1p(y_train)

In [73]:
y_train_log

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,1.252763,2.564949,5.128122
1,1.131402,2.388763,4.604170
2,1.163151,2.322388,4.852811
3,1.163151,2.370244,5.182907
4,0.916291,2.001480,4.810557
...,...,...,...
7105,0.788457,1.774952,5.356586
7106,0.832909,1.808289,5.258016
7107,0.875469,1.916923,5.404028
7108,0.955511,1.824549,5.431098


In [74]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [75]:
reg = RandomForestRegressor()

In [76]:
model =  MultiOutputRegressor(reg).fit(X_train, y_train_log)

In [77]:
y_pred = model.predict(X_test)

In [78]:
y_pred = np.expm1(y_pred)

In [79]:
y_pred

array([[  1.85879498,   5.21253511, 236.46469415],
       [  2.38769542,   7.36300934, 280.88761557],
       [  2.20482174,   7.39473545, 316.16734653],
       ...,
       [  2.45245584,  12.33235381, 374.9122981 ],
       [  2.05890001,  11.16080333, 294.3564242 ],
       [  2.08540362,  12.19145737, 291.28688215]])

In [80]:
sub_df=pd.read_csv('sample_submission.csv')
datasets=pd.concat([sub_df['date_time'],y_pred],axis=1)
datasets.columns=['date_time',"target_carbon_monoxide","target_benzene","target_nitrogen_oxides"]
datasets.to_csv('sample_submission.csv',index=False)

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [81]:
df_submission = pd.read_csv("sample_submission.csv")

In [82]:
df_submission.drop(["target_carbon_monoxide","target_benzene","target_nitrogen_oxides"],axis=1,inplace=True)

In [83]:
df_submission["target_carbon_monoxide"] = y_pred[:,0]

In [84]:
df_submission["target_benzene"] = y_pred[:,1]

In [85]:
df_submission["target_nitrogen_oxides"] = y_pred[:,2]

In [86]:
df_submission.head()

,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,1.858795,5.212535,236.464694
1,2011-01-01 01:00:00,2.387695,7.363009,280.887616
2,2011-01-01 02:00:00,2.204822,7.394735,316.167347
3,2011-01-01 03:00:00,2.317906,8.513702,367.510624
4,2011-01-01 04:00:00,1.527850,6.847152,253.114557


In [87]:
df_submission.shape

(2247, 4)

In [88]:
df_submission.to_csv('sample_submission6.csv',index=False)

In [ ]:
df_train['date_time']= pd.to_datetime(df_train['date_time'])

In [ ]:
df_test['date_time']= pd.to_datetime(df_test['date_time'])

In [ ]:
df_train.info()

In [ ]:
df_train1= create_ml_features(df_train.copy())

In [ ]:
df_train1.reset_index(drop=False, inplace=True)

In [ ]:
df_train1.head()

In [ ]:
df_train.info()

In [ ]:
df_train1['date_time'] = df_train1['date_time'].view('datetime64[ns]').view(np.int64)/10**9


In [ ]:
df_train1.head()

In [ ]:
X_train= df_train1.iloc[:5646,:]
X_test = df_train1.iloc[5647:,:]

In [ ]:
len(X_train),len(X_test)

In [ ]:
df_train.columns

In [ ]:
y = df_train.iloc[:,8:].copy()

In [ ]:
y.reset_index(drop=True, inplace=True)

In [ ]:
y.drop('date_time',axis=1,inplace=True)

In [ ]:
y_log= np.log1p(y)

In [ ]:
y_log_train = y_log[:5646]
y_log_test = y_log[5647:]

In [ ]:
len(y_log_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
y_log_train

In [ ]:
reg = RandomForestRegressor()

In [ ]:
model =  MultiOutputRegressor(reg).fit(X_train, y_log_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred = np.expm1(y_pred)

In [ ]:
y_test = np.expm1(y_log_test)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mean_absolute_percentage_error(y_test, y_pred)

In [ ]:

y_test['target_carbon_monoxide']

In [ ]:
df_train2= create_ml_features(df_train.copy())
df_test2= create_ml_features(df_test.copy())

In [ ]:
df_train2

In [ ]:
df_train2.reset_index(drop=False, inplace=True)
df_test2.reset_index(drop=False, inplace=True)

In [ ]:
df_train2['date_time'] = df_train2['date_time'].astype('datetime64[ns]').astype(np.int64)/10**5
df_test2['date_time'] = df_test2['date_time'].astype('datetime64[ns]').astype(np.int64)/10**5

In [ ]:
df_train2.shape

In [ ]:
df_test2.shape

In [ ]:
df_train.drop(df_train.tail(1).index,inplace=True)

In [ ]:
df_test.shape

In [ ]:
X_train = df_train2.iloc[:,:]
y_train =df_train.iloc[:,8:].copy()
X_test = df_test2.iloc[:,:]


In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train.reset_index(drop=True, inplace=True)

In [ ]:
X_test.shape

In [ ]:
df_train2

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
df_train2.columns

In [ ]:
df_test2.columns

In [ ]:
y_train_log = np.log1p(y_train)

In [ ]:
y_train_log

In [ ]:
model1 =   MultiOutputRegressor(reg).fit(X_train, y_train_log)

In [ ]:
y_train_log['target_carbon_monoxide']

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
X_train['date_time']

In [ ]:
X_test.info()

In [ ]:
X_test['date_time'].astype

In [44]:
np.any(np.isnan(y_train))

False

In [ ]:
y_pred = model1.predict(X_test)

In [ ]:
import numpy.distutils.system_info as sysinfo
sysinfo.platform_bits
    

In [ ]:
import sklearn.distutils.system_info as sysinfo
sysinfo.platform_bits
    